# Arranquemos! 
Lo primero que vamos a hacer es crear un (listening) socket TCP. cuando llamamos a la función *socket* le vamos a pasar como parámetro "SOCK_STREAM" para lograr esto.

In [1]:
from socket import *
import sys 

serverSocket = socket(AF_INET, SOCK_STREAM)
print(serverSocket)

<socket.socket fd=940, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0>


# Ahora preparemos el socket...
Para esto tenemos que primero, asignarle un número de puerto y dirección IP. Revisen el cheatsheet!
Luego vamos a hacer que nuestro programa de Python _escuche_ en el socket.

In [2]:
#COMPLETAR (son 2 líneas solamente ;) )
direccion = ""
puerto = 2190
serverSocket.bind((direccion, puerto))
serverSocket.listen(1) # preparamos el socket para que espere mensajes
print(serverSocket)

# si nos tira error es porque no se puede usar el mismo socket dos veces!

<socket.socket fd=940, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('0.0.0.0', 2190)>


# Listo! Ahora vamos a tener que quedarnos escuchando en el server hasta que alguien nos mande algo...
Con lo que hicimos hasta ahora, si le llega a nuestro SO un paquete con cierta dirección IP y nro de puerto, nos llega a nosotros. Recordemos que:
- Debemos quedarnos "esperando" hasta que nos llegue un paquete.
- Si nos llega alguno, es algun usuario que quiere conectarse. En ese caso, necesitamos un socket nuevo para esta conexión! Noten que en el código de abajo, se hace esto en la tercera línea
- Una vez que abramos un nuevo socket (connection socket), hay que esperar que nos llegue un mensaje...

En el código de abajo faltan resolver algunas cosas.
1) Hay que encargarse de leer el mensaje que recibirá el connection socket.
2) Hay que encargarse de resolver que recurso nos están enviando (por ejemplo, información guardada en un txt).
3) En caso de encontrar el recurso pedido, hay que enviar primero un OK y después el recurso en sí.
4) En caso de que no esté el recurso pedido, hay que enviar un mensaje de error. ¿Cuál es?

Revisen el código con cuidado. ¿Cómo pueden testear lo que hicieron?

In [3]:
# hacemos los mensajes de respuesta
import os
header_ok = "HTTP/1.1 200 OK\r\n\r\n".encode()
header_error = "HTTP/1.1 404 Not Found\r\n\r\n".encode()


while True:
    print('Esperemos a que nos llegue un mensaje...')
    connectionSocket, addr = serverSocket.accept() #Si nos llega un pedido de conexión, con esta línea la aceptamos.
    print("Nos pidieron una conexión! Abriendo socket para: ", connectionSocket)

    try:
        message = connectionSocket.recv(1024).decode() #Recibir el mensaje
        print(message)
        filename = message.split()[1] # Resolver qué recurso que pide el mensaje
        print(filename)
        print(os.listdir())
        f = open(filename[1:]) # queda solo el nombre del request
        data_respuesta = f.read() # Cargar el recurso
        print(data_respuesta)

        #Enviar el header HTTP de la respuesta
        connectionSocket.send(header_ok)

        #Las siguientes líneas envian el contenido de la respuesta
        for i in range(0, len(data_respuesta)):
            connectionSocket.send(data_respuesta[i].encode())
            connectionSocket.send("\r\n".encode())
        connectionSocket.close()

    except IOError:
        #En caso de no tener el archivo, enviar mensaje HTTP correspondiente
        connectionSocket.send(header_error) 
        connectionSocket.send("We cannot find the file".encode())
        connectionSocket.send("\r\n".encode())

        connectionSocket.close()

Esperemos a que nos llegue un mensaje...
Nos pidieron una conexión! Abriendo socket para:  <socket.socket fd=924, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 2190), raddr=('127.0.0.1', 53431)>
GET /prueba.txt HTTP/1.1
Host: localhost:2190
Connection: keep-alive
Cache-Control: max-age=0
sec-ch-ua: "Google Chrome";v="123", "Not:A-Brand";v="8", "Chromium";v="123"
sec-ch-ua-mobile: ?0
sec-ch-ua-platform: "Windows"
Upgrade-Insecure-Requests: 1
User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36
Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7
Sec-Fetch-Site: cross-site
Sec-Fetch-Mode: navigate
Sec-Fetch-User: ?1
Sec-Fetch-Dest: document
Accept-Encoding: gzip, deflate, br, zstd
Accept-Language: es-US,es;q=0.9,en-GB;q=0.8,en;q=0.7,es-419;q=0.6
Cookie: _xsrf=2|86339507|aed6bc9c6000c8635

IndexError: list index out of range